# get size

In [ ]:
import sys
import os

In [1]:
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)




In [1]:
import sys
def sizeof_fmt2(df):
    suffix = 'B'
    num = sys.getsizeof(df)
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

# save to aws
- helper function for saving dataframes to S3

In [1]:
import os
import boto3
import botocore
import pandas as pd
from pandas import DataFrame
import string, random
from urllib.parse import urlparse
import re

def save_df_to_s3(df, path_to_directory, compression='gzip', encoding = None):
    # parse url
    # the default is s3://awsant-lalaport-data
    o = urlparse(path_to_directory)
    scheme = 's3' if ((o.scheme == '') | (o.scheme == 's3n')) else o.scheme
    mybucket = o.netloc if o.netloc != '' else 'awsant-lalaport-data'
    # remove heading '/' if there is any
    path_to_directory = re.sub('^/', '', o.path)
    
    # make conncection
    s3 = boto3.resource(scheme)
    bucket = s3.Bucket(mybucket)
    
    # where am I writing to ?
    print("saving to s3://" + mybucket + "/" + path_to_directory)
    
    # write in tempfile
    tempfile = ''.join(random.choices(string.ascii_uppercase + string.digits, k=20)) + ".gz"
    df.to_csv(tempfile, compression = compression, encoding = encoding)
    s3.Object(mybucket, path_to_directory).put(Body=open(tempfile, 'rb'))
    
    # erase tempfile
    os.remove(tempfile)
    
    print("saved! to s3://" + mybucket + "/" + path_to_directory)

In [ ]:
import os
import boto3
import botocore
import pandas as pd
from pandas import DataFrame
import string, random

def save_df_to_s3_csv(df, path_to_directory):
    # make conncection
    s3 = boto3.resource('s3')
    mybucket = 'awsant-lalaport-data'
    bucket = s3.Bucket(mybucket)
    
    # where am I writing to ?
    print("saving to s3://" + mybucket + "/" + path_to_directory)
    
    # write in tempfile
    tempfile = ''.join(random.choices(string.ascii_uppercase + string.digits, k=20)) + ".csv"
    df.to_csv(tempfile)
    s3.Object(mybucket, path_to_directory).put(Body=open(tempfile, 'rb'))
    
    # erase tempfile
    os.remove(tempfile)

In [ ]:
import os
import boto3
import botocore
import pandas as pd
from pandas import DataFrame
import string, random
from urllib.parse import urlparse


def save_object_to_s3(object_to_save, path_to_directory):
    # parse url
    # the default is s3://awsant-lalaport-data
    o = urlparse(path_to_directory)
    scheme = 's3' if ((o.scheme == '') | (o.scheme == 's3n')) else o.scheme
    mybucket = o.netloc if o.netloc != '' else 'awsant-lalaport-data'
    path_to_directory =o.path[1:]
    
    # make conncection
    s3 = boto3.resource(scheme)
    bucket = s3.Bucket(mybucket)
    
    # where am I writing to ?
    print("saving to s3://" + mybucket + "/" + path_to_directory)
    
    # put file
    s3.Object(mybucket, path_to_directory).put(Body=object_to_save)
    
    # erase file
    os.remove(object_to_save)
    
    print("saved! to s3://" + mybucket + "/" + path_to_directory)    

In [4]:
import s3io
from sklearn.externals import joblib
from urllib.parse import urlparse
import boto

def joblib_dump_s3(object_to_save, path_to_directory):
    # parse url
    # the default is s3://awsant-lalaport-data
    o = urlparse(path_to_directory)
    scheme = 's3' if ((o.scheme == '') | (o.scheme == 's3n')) else o.scheme
    bucket = o.netloc if o.netloc != '' else 'awsant-lalaport-data'
    key = re.sub('^/', '', o.path)
    
    #s3 connection
    s3 = boto.connect_s3()
    
    # open connection and save
    with s3io.open('s3://{0}/{1}'.format(bucket, key), mode = 'w',  s3_connection=s3) as s3_file:
        joblib.dump(object_to_save, s3_file, compress = True)
    
    print("saved to s3://" + bucket + "/" + key) 
        
def joblib_load_s3(path_to_directory):
    # parse url
    # the default is s3://awsant-lalaport-data
    o = urlparse(path_to_directory)
    scheme = 's3' if ((o.scheme == '') | (o.scheme == 's3n')) else o.scheme
    bucket = 'awsant-lalaport-data' if o.netloc == '' else o.netloc
    key = re.sub('^/', '', o.path)
    
    #s3 connection
    s3 = boto.connect_s3()
    
    print('Loading from s3://' + bucket + '/' + key)
    
    with s3io.open('s3://{0}/{1}'.format(bucket, key), mode = 'r', s3_connection=s3) as s3_file:
        return joblib.load(s3_file)

# print

In [4]:
def print_unique_counts(df):
    for c in df.columns:
        print ("------------------ %s -----------------" % c)
        print (df[c].nunique())

In [3]:
def print_value_counts(df, nrows = 15):
    for c in df.columns:
        print ("------------------ %s -----------------" % c)
        print (df[c].value_counts().head(nrows))

# filter SCs

## shopping around

In [2]:
import pandas as pd
import os
from pathlib import Path
import git
repo = git.Repo('.', search_parent_directories=True)
module_path = repo.working_tree_dir

def filter_shopping_around_scs(df, module_path = module_path):

    # get master
    master_path = Path(module_path) / 'master/ms_sc.tab'
    master = pd.read_table(master_path)
    master = master[pd.isnull(master.type) != True]

    # get sc to use
    ms_sc_path = Path(module_path) / 'master/MS_SC_v5.csv'
    sc_to_use = pd.read_csv(ms_sc_path, skiprows=1, usecols=master.column)

    # get those to use in shopping around
    sc_to_use = sc_to_use[sc_to_use['ShoppingAround'] == 'Use']
    
    # change type
    sc_to_use['SC_CODE'] = sc_to_use['SC_CODE'].astype('int64')
    sc_to_use['SC_CODE_NEW'] = sc_to_use['SC_CODE_NEW'].astype('int64')
    
    # get SC code new
    new_sc_codes = sc_to_use[['SC_CODE','SC_CODE_NEW']]
    
    # merge new sc codes
    df = df.merge(new_sc_codes)
    
    # filter SC codes in those to use
    df = df[df['SC_CODE_NEW'].isin(new_sc_codes['SC_CODE_NEW'])]
    
    # drop old sc
    del df['SC_CODE']
    
    # rename sc codes
    df.rename(index=str, columns = {'SC_CODE_NEW':'SC_CODE'}, inplace = True)
    
    return df

## campaign

In [2]:
import pandas as pd
import os
from pathlib import Path

import git
repo = git.Repo('.', search_parent_directories=True)
module_path = repo.working_tree_dir

def filter_campaign_scs(df, module_path):
    # delete new sc codes
    if 'SC_CODE_NEW' in df:
        del df['SC_CODE_NEW']

    # get sc to use
    ms_sc_path = Path(module_path) / 'master/MS_SC_v7.xlsx'
    sc_to_use = pd.read_excel(ms_sc_path, skiprows=1)

    # get those to use in shopping around
    sc_to_use = sc_to_use.loc[sc_to_use['CampaignAnalysis'] == 'Use',:]
    
    # change type
    sc_to_use['SC_CODE'] = sc_to_use['SC_CODE'].astype('int64')
    sc_to_use['SC_CODE_NEW'] = sc_to_use['SC_CODE_NEW'].astype('int64')
    
    # get SC code new
    new_sc_codes = sc_to_use.loc[:,['SC_CODE','SC_CODE_NEW']]
    
    # merge new sc codes
    df = df.merge(new_sc_codes)
    
    # filter SC codes in those to use
    df = df.loc[df['SC_CODE_NEW'].isin(new_sc_codes['SC_CODE_NEW']),:]
    
    # drop old sc
    del df['SC_CODE']
    
    # rename sc codes
    df.rename(index=str, columns = {'SC_CODE_NEW':'SC_CODE'}, inplace = True)
    
    return df

# lift chart

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
from pathlib import Path

def print_lift_chart(actual, probability, title, num_bins = 10, 
                     save_file = False, module_path = None, file_name = None,
                     save_summary = False, summary_save_path = None):
    # make data
    d = {'actual':actual, 'probability':probability}
    df = pd.DataFrame(d)
    
    # get mean probability
    mean_probability = df.probability.agg('mean')
    mean_actual = df.actual.agg('mean')
    
    # add bin
    df['bins'] = pd.qcut(df.probability, num_bins, duplicates='drop')
    
    # summarize on bin
    summary = df.groupby('bins').agg('mean')
    summary['Bins'] = range(1,len(summary.index)+1)
    
    # plot
    ax = sns.pointplot(x = 'Bins', y = 'actual', data = summary,
                  color = 'grey')

    # add title
    ax.set_title(title)

    # add mean
    ax.plot([0,num_bins],[mean_actual,mean_actual], linewidth=2)
    
    # set ylim
    ax.set(ylim=(0, None))
    
    # save plot
    if save_file:
        save_path = Path(module_path).parent / 'share/' / Path(file_name)
        sns.plt.savefig(str(save_path))
        
    if save_summary:
        save_df_to_s3_csv(summary, summary_save_path)
    
    return ax

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd

def print_lift_chart_comparison(actual, probability, target, version, num_bins = 10, save_file = False, module_path = None, file_name = None):
    # make data
    d = {'actual':actual, 'probability':probability}
    df = pd.DataFrame(d)
    
    # get mean probability
    mean_probability = df.probability.agg('mean')
    mean_actual = df.actual.agg('mean')
    
    # add bin
    df['bins'] = pd.qcut(df.probability, num_bins, duplicates='drop')
    
    # summarize on bin
    summary = df.groupby('bins').agg('mean')
    summary['n'] = range(1,len(summary.index)+1)
    
    # melt
    melted_summary = pd.melt(summary, id_vars = 'n', value_vars=['actual', 'probability'])
    
    # plot comparison
    ax = sns.pointplot(x = 'n', y = 'value', hue = 'variable', 
                       data = melted_summary,
                       markers = ['o','o'],
                       linestyles = ['--','--']
                      )
    # set title
    ax.set_title('Comparison of actual and probability for SC' + str(target) + ', ' + version)
    
    # set ylim
    ax.set(ylim=(0, None))

    # add mean
    plt.plot([0,num_bins],[mean_actual,mean_actual], linewidth=2)
    
    # save plot
    if save_file:
        save_path = Path(module_path).parent / 'share/' / Path(file_name)
        sns.plt.savefig(str(save_path))
    
    return ax

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
from pathlib import Path

def print_lift_chart_variables(actual, probability, variable, title, num_bins = 10,  save_file = False, module_path = None, file_name = None):
    # make data
    d = {'actual':actual, 'probability':probability, 'variable':variable}
    df = pd.DataFrame(d)
    
    # get mean probability
    mean_actual = df.loc[df.variable == 'test',:].actual.mean()
    
    # initialzie
    temp2 = df.head(0)
    
    # add bin according to category
    for cat in df['variable'].unique():
        print(cat)
        temp = df.loc[df['variable'] == cat,:].copy()
        print(temp.shape)

        temp['bins'] = pd.qcut(temp['probability'], num_bins, labels = range(1, num_bins+1))
        temp2 = pd.concat([temp2, temp], ignore_index=True)
    
    df = temp2
    
    # summarize on bin
    summary = df.groupby(['variable', 'bins']).agg('mean')
    summary.reset_index(inplace = True)
    
    # get score
    summary_test = summary.loc[summary.variable == 'test', :].copy()
    summary_test['score'] = 100 / num_bins * summary_test.bins
    score = '{:.2f}'.format(sum(summary_test.actual * summary_test.score) / sum(summary_test.actual))
    
    # plot
    ax = sns.pointplot(x = 'bins', y = 'actual', hue = 'variable', data = summary)

    # add title
    ax.set_title(title)
    
    # add score
    ax.set_xlabel('Score: ' + score)

    # add mean
    ax.plot([0,num_bins],[mean_actual,mean_actual], linewidth=2)
    
    # set ylim
    ax.set(ylim=(0, None))

    # save plot
    if save_file:
        save_path = Path(module_path).parent / 'share/' / Path(file_name)
        sns.plt.savefig(str(save_path))    
    
    return ax